<a href="https://colab.research.google.com/github/anvy87/Kite-connect/blob/main/kc_pivotprice_intraday.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install kiteconnect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.5/771.5 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.4/264.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.6 MB/s eta 0:00:00


In [2]:
from kiteconnect import KiteConnect
import pandas as pd
import numpy as np
import logging
import os
import time
import datetime as dt
api_key = "1ll4p8ziqlk8hh7g"
api_secret = "x59r5u1vd25annfpgts22wv8ykfueirl"
kite = KiteConnect(api_key=api_key)

In [3]:
print(kite.login_url()) #use this url to manually login and authorize yourself

https://kite.zerodha.com/connect/login?api_key=1ll4p8ziqlk8hh7g&v=3


In [8]:
#generate trading session
request_token = "4Zd8EtDOuCh0b3wwT2UmG2BUr6iT7u4X" #Extract request token from the redirect url obtained after you authorize yourself by loggin in
data = kite.generate_session(request_token, api_secret=api_secret)
#create kite trading object
kite.set_access_token(data["access_token"])

In [9]:
#get dump of all NSE instruments
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv("NSE_Instruments.csv",index=False)

In [4]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [5]:
def fetchOHLC(ticker,interval,duration):
  """extracts historcial data and outputs in the form of dataframe"""
  instrument = instrumentLookup(instrument_df,ticker)
  data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration),dt.date.today(),interval))
  # data1 = pd.DataFrame(kite.historical_data(instrument,dt.date.today(),dt.date.today(),interval))
  # print(data1.head())
  data.set_index("date",inplace=True)
  return data

In [6]:
# List of Nifty 50 constituent trading symbols (update as per latest list)
nifty50_symbols = ["RELIANCE", "HDFCBANK", "BHARTIARTL", "TCS", "ICICIBANK", "SBIN", "INFY", "HINDUNILVR", "BAJFINANCE", "ITC"]

In [7]:
instrument_token = [6401, 40193, 60417, 81153, 98049, 119553, 177665, 225537, 315393, 341249, 345089, 348929, 408065, 424961, 492033, 502785, 519937,
 633601, 738561, 779521, 857857, 878593, 884737, 895745, 897537, 969473, 1270529, 1304833, 1346049, 1510401, 1850625, 2815745, 2939649,
 2952193, 2953217, 2977281, 3001089, 3465729, 4267265, 4268801, 4598529, 4644609, 5215745, 5582849, 3861249]

# **placing order**

In [10]:
def placeMarketOrder(symbol, buy_sell, quantity):
  # place intraday market order on NSE
  if buy_sell == "buy":
    t_type = kite.TRANSACTION_TYPE_BUY
  elif buy_sell == "sell":
    t_type = kite.TRANSACTION_TYPE_SELL
  kite.place_order(tradingsymbol=symbol,
                   exchange=kite.EXCHANGE_NSE,
                   transaction_type=t_type,
                   quantity=5*quantity, # 5x leverage
                   order_type=kite.ORDER_TYPE_MARKET,
                   product=kite.PRODUCT_MIS,
                   variety=kite.VARIETY_REGULAR)

In [ ]:
df.iloc[20, : ]['close']

np.float64(830.9)

In [ ]:
df_yesterday = fetchOHLC("SBIN","5minute", 3)
print("SBIN", df_yesterday.head(30))
close_11am = df.iloc[20, : ]['close']
print(close_11am)
df_today = fetchOHLC("SBIN","5minute",0)
print(df_today.head())
intraday_high = df_today['high'].max()
print(intraday_high)
intraday_low = df_today['low'].min()
print(intraday_low)
capital = 10000 #position size
quantity = int(capital/df_today["close"][-1])
'''instrument = instrumentLookup(instrument_df, i)
if instrument != -1: # Check if instrument token was found
    yesterday_close = kite.quote([instrument])[str(instrument)]['ohlc']['close'] # Use instrument token as key
    #print(close_11am)
    #print(f"{i}: {yesterday_close}") # Print the symbol and its close price
else:
    print(f"Instrument token not found for {i}")'''
pivot_price = (intraday_high+intraday_low+yesterday_close)/3
#print(pivot_price, close_11am)
if close_11am > pivot_price:
  print(f"buy SBIN")
  #placeMarketOrder(i, "buy", quantity)
elif close_11am < pivot_price:
  print(f"sell SBIN")
  #placeMarketOrder(i, "sell", quantity)

In [14]:
import warnings
warnings.filterwarnings('ignore')  # Suppress all warnings

for i in nifty50_symbols:
  #df_yesterday = fetchOHLC(i,"5minute", 1)
  #print(i, df)
  df_today = fetchOHLC(i, "5minute", 0)
  capital = 10000 #position size
  quantity = int(capital/df_today["close"][-1])
  close_11am = df_today['close'][-1]
  intraday_high = df_today['high'].max()
  intraday_low = df_today['low'].min()
  instrument = instrumentLookup(instrument_df, i)
  if instrument != -1: # Check if instrument token was found
      yesterday_close = kite.quote([instrument])[str(instrument)]['ohlc']['close'] # Use instrument token as key
  else:
      print(f"Instrument token not found for {i}")
  pivot_price = (intraday_high+intraday_low+yesterday_close)/3
  print(f'pivot_price is {pivot_price}, close@11 is {close_11am}')
  if close_11am > pivot_price:
    print(f"buy {i}")
    #placeMarketOrder(i, "buy", quantity)
  elif close_11am < pivot_price:
    print(f"sell {i}")
    #placeMarketOrder(i, "sell", quantity)

pivot_price is 1396.2333333333333, close@11 is 1399.3
buy RELIANCE
pivot_price is 968.1333333333332, close@11 is 966.1
sell HDFCBANK
pivot_price is 1905.3, close@11 is 1907.5
buy BHARTIARTL
pivot_price is 3113.9666666666667, close@11 is 3112.4
sell TCS
pivot_price is 1420.8333333333333, close@11 is 1418.0
sell ICICIBANK
pivot_price is 824.5500000000001, close@11 is 824.0
sell SBIN
pivot_price is 1512.3666666666668, close@11 is 1509.4
sell INFY
pivot_price is 2580.9333333333334, close@11 is 2582.9
buy HINDUNILVR
pivot_price is 1011.2166666666667, close@11 is 1011.7
buy BAJFINANCE
pivot_price is 412.8999999999999, close@11 is 412.8
sell ITC
